## My research question and Corpus

My research question is "How can bots be identified in social media comments?".

My corpus is list of comments from Twitter/X. 

I collected first 100 comments from Donald Trump's recent tweet https://x.com/realDonaldTrump/status/1894126415932526802/photo/1 as well as, got detailed information about comment authors. In addition, I got comments from russian media "Meduza" from the tweet https://x.com/meduzaproject/status/1894022390490837112 

Comments collected by endpoint: https://api.twitter.com/2/tweets/search/recent

UserInfo collected by endpoint: https://api.twitter.com/2/users?ids={batch_str}&user.fields=username,name,created_at,location,verified


Getting data through Twitter API is very unconvineant if you use free API.
Available pull size for tweets and retweets is just 100 per month.
My first try was failed and I reached out my monthly limit.
Collecting 160 retweets from 2 different tweets needed 3 different accounts from me)))

I still have no idea how can I detect and which technicues I will use. However, I found repository with different datasets which I think will be helpfull if I will use supervised learning. 



In [1]:
import requests
import json
import time

In [43]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAABpFzgEAAAAAsPyyGIzE8c7x0o22RQvuwoHcars%3Djg0EW2CqUZ8rEduWfKUGJySMPayi4I0fke4YoJgDMNgKt1U67l"

## Getting tweet comments by conversation id

In [41]:
TWEET_ID = "1894126415932526802"
FILE_PATH = f"data/tweet_info_{TWEET_ID}.json"

def get_tweet_replies(tweet_id):
    url = "https://api.twitter.com/2/tweets/search/recent"
    headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}
    
    all_replies = []
    next_token = None

    while True:
        params = {
            "query": f"conversation_id:{tweet_id}",
            "tweet.fields": "author_id,created_at",
            "expansions": "author_id",
            "user.fields": "username",
            "max_results": 100,
        }

        if next_token:
            params["next_token"] = next_token  

        response = requests.get(url, headers=headers, params=params, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            with open(FILE_PATH+"_all", "w", encoding="utf-8") as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
            all_replies.extend(data.get("data", []))

            
            next_token = data.get("meta", {}).get("next_token")
            if not next_token:  
                break  

        else:
            print(f"Error {response.status_code}: {response.json()}")
            break
    
    return all_replies

'''
tweet_replies = get_tweet_replies(TWEET_ID)


with open(FILE_PATH, "w", encoding="utf-8") as file:
    json.dump(tweet_replies, file, ensure_ascii=False, indent=4)

print(f"Data saved to {FILE_PATH}")
'''

'\ntweet_replies = get_tweet_replies(TWEET_ID)\n\n\nwith open(FILE_PATH, "w", encoding="utf-8") as file:\n    json.dump(tweet_replies, file, ensure_ascii=False, indent=4)\n\nprint(f"Data saved to {FILE_PATH}")\n'

## Get Users info by ID

In [44]:

FILE_PATH = f"data/tweet_info_{TWEET_ID}.json"  
FILE_PATH1 = f"data/user_info_{TWEET_ID}.json"  
BATCH_SIZE = 100  
SLEEP_TIME = 900  

with open(FILE_PATH, "r", encoding="utf-8") as file:
    data = json.load(file)


users = list(set(tweet["author_id"] for tweet in data))
print(f"Total unique users: {len(users)}")


def get_users_info(user_ids):
    """Fetches user information from Twitter in batches with 15-minute wait time."""
    all_users_info = []

    for i in range(0, len(user_ids), BATCH_SIZE):
        batch = user_ids[i : i + BATCH_SIZE]  
        batch_str = ",".join(batch)  

        url = f"https://api.twitter.com/2/users?ids={batch_str}&user.fields=username,name,created_at,location,verified"
        headers = {"Authorization": f"Bearer {BEARER_TOKEN}"}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            with open(FILE_PATH1+"_all", "w", encoding="utf-8") as file:
                json.dump(response.json(), file, ensure_ascii=False, indent=4)
            all_users_info.extend(response.json().get("data", []))
        else:
            print(f"Error {response.status_code}: {response.json()}")

        if i + BATCH_SIZE < len(user_ids):  
            print("Rate limit reached. Waiting 15 minutes before the next batch...")
            time.sleep(SLEEP_TIME)  

    return all_users_info


users_info = get_users_info(users)


with open(FILE_PATH1, "w", encoding="utf-8") as file:
    json.dump(users_info, file, ensure_ascii=False, indent=4)

print(f"User data saved to {FILE_PATH1}")


Total unique users: 84
User data saved to data/user_info_1894126415932526802.json
